In [1]:
import pandas as pd
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from firebase_admin import messaging
import time

import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
# from sklearn.externals ㅌimport joblib
import joblib
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
#matplotlib inline

from numpy.random import seed
# from tensorflow import set_random_seed
# from tensorflow import random_set_seed
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers
import timeit
from tensorflow.keras.models import load_model
model = load_model('model_137_0604_t20.h5')

cred=credentials.Certificate('mykey.json')
firebase_admin.initialize_app(cred,{
    'databaseURL':'https://sensor-3b995-default-rtdb.firebaseio.com/'
})

In [2]:
merged_train = pd.read_csv("data/data_137_0604_train.csv")
merged_train.set_index('date', inplace=True)

In [3]:
def send_to_firebase_cloud_messaging():
    # This registration token comes from the client FCM SDKs.
    registration_token = 'cb-4KYtaRViPGhx5tuiFkJ:APA91bEsH031YyrWhjhkpaK-npOcTrRbUo9oMOGRWV3-BHE9ishylZe3yZkyL7WWeXCo9feiDvjn1CGSoJxXStcbxr_7bf_mxoCi6GekqUCzHVw9wKOtzFi-a57tkqXK24XbuCHqTfwr'
    # See documentation on defining a message payload.
    message = messaging.Message(
    notification=messaging.Notification(
        title='안녕하세요 타이틀 입니다',
        body='안녕하세요 메세지 입니다',
    ),
    token=registration_token,
    )

    response = messaging.send(message)
    # Response is a message ID string.
    print('Successfully sent message:', response)
#     send_to_firebase_cloud_messaging()

In [4]:
def Test_MakeSlidingWindows(origin_test, test, lookback, gtruthName='SPI'):
    test_df = pd.DataFrame(test, index=origin_test.index) 
    test_df.columns = origin_test.columns 
    column_list = list(test_df)
    
    for s in range(1, lookback+1):
        tmp_test = test_df[column_list].shift(s)
        tmp_test.columns = "shift_" + tmp_test.columns + "_" + str(s) 
        test_df[tmp_test.columns] = test_df[column_list].shift(s)
    return test_df

In [ ]:
while True :
    
    start_time = timeit.default_timer()
    ref = db.reference()
    ref = ref.get()
    date = []
    data = []
    character = '[]'
    
    for key,value in ref.items():
        date.append(key)
        for x in range(len(character)):
            value = value.replace(character[x],"")
        data.append(value) 

    date = date[-30:]    
    data = data[-30:]      
        
    ex_cols = ['val']
    cols = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41','42','43','44','45','46','47','48','49','50','51','52','53','54','55','56','57','58','59','60','61','62','63','64']
    
    merged_data = pd.DataFrame(data,index=date,columns=ex_cols)
    merged_data=merged_data['val'].str.split(',',expand=True)
    
    merged_invade = pd.DataFrame(merged_data.values,index=date,columns=cols)
      
    origin_train = merged_train.copy()
    origin_test = merged_invade.copy()
    lookback = 19
    
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(origin_train)
    X_test = scaler.transform(origin_test)
    scaler_filename = "scaler_data"
    joblib.dump(scaler, scaler_filename)
    
    tmp_test = origin_test.copy()

    test = Test_MakeSlidingWindows(origin_test,origin_test,lookback,gtruthName='SPI')
    X_test = Test_MakeSlidingWindows(tmp_test, X_test, lookback, gtruthName='SPI')
    
#     cols = ['a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10', 'a11', 'a12', 'a13', 'a14', 'a15', 'a16', 'a17', 'a18', 'a19', 'a20', 'a21', 'a22', 'a23', 'a24', 'a25', 'a26', 'a27', 'a28', 'a29', 'a30', 'a31', 'a32', 'a33', 'a34', 'a35', 'a36', 'a37', 'a38', 'a39', 'a40', 'a41', 'a42', 'a43', 'a44', 'a45', 'a46', 'a47', 'a48', 'a49', 'a50', 'a51', 'a52', 'a53', 'a54', 'a55', 'a56', 'a57', 'a58', 'a59', 'a60', 'a61', 'a62', 'a63', 'a64', 
#         'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10', 'b11', 'b12', 'b13', 'b14', 'b15', 'b16', 'b17', 'b18', 'b19', 'b20', 'b21', 'b22', 'b23', 'b24', 'b25', 'b26', 'b27', 'b28', 'b29', 'b30', 'b31', 'b32', 'b33', 'b34', 'b35', 'b36', 'b37', 'b38', 'b39', 'b40', 'b41', 'b42', 'b43', 'b44', 'b45', 'b46', 'b47', 'b48', 'b49', 'b50', 'b51', 'b52', 'b53', 'b54', 'b55', 'b56', 'b57', 'b58', 'b59', 'b60', 'b61', 'b62', 'b63', 'b64', 
#         'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'c15', 'c16', 'c17', 'c18', 'c19', 'c20', 'c21', 'c22', 'c23', 'c24', 'c25', 'c26', 'c27', 'c28', 'c29', 'c30', 'c31', 'c32', 'c33', 'c34', 'c35', 'c36', 'c37', 'c38', 'c39', 'c40', 'c41', 'c42', 'c43', 'c44', 'c45', 'c46', 'c47', 'c48', 'c49', 'c50', 'c51', 'c52', 'c53', 'c54', 'c55', 'c56', 'c57', 'c58', 'c59', 'c60', 'c61', 'c62', 'c63', 'c64', 
#         'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'd10', 'd11', 'd12', 'd13', 'd14', 'd15', 'd16', 'd17', 'd18', 'd19', 'd20', 'd21', 'd22', 'd23', 'd24', 'd25', 'd26', 'd27', 'd28', 'd29', 'd30', 'd31', 'd32', 'd33', 'd34', 'd35', 'd36', 'd37', 'd38', 'd39', 'd40', 'd41', 'd42', 'd43', 'd44', 'd45', 'd46', 'd47', 'd48', 'd49', 'd50', 'd51', 'd52', 'd53', 'd54', 'd55', 'd56', 'd57', 'd58', 'd59', 'd60', 'd61', 'd62', 'd63', 'd64', 
#         'e1', 'e2', 'e3', 'e4', 'e5', 'e6', 'e7', 'e8', 'e9', 'e10', 'e11', 'e12', 'e13', 'e14', 'e15', 'e16', 'e17', 'e18', 'e19', 'e20', 'e21', 'e22', 'e23', 'e24', 'e25', 'e26', 'e27', 'e28', 'e29', 'e30', 'e31', 'e32', 'e33', 'e34', 'e35', 'e36', 'e37', 'e38', 'e39', 'e40', 'e41', 'e42', 'e43', 'e44', 'e45', 'e46', 'e47', 'e48', 'e49', 'e50', 'e51', 'e52', 'e53', 'e54', 'e55', 'e56', 'e57', 'e58', 'e59', 'e60', 'e61', 'e62', 'e63', 'e64', 
#         'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 
#         'g1', 'g2', 'g3', 'g4', 'g5', 'g6', 'g7', 'g8', 'g9', 'g10', 'g11', 'g12', 'g13', 'g14', 'g15', 'g16', 'g17', 'g18', 'g19', 'g20', 'g21', 'g22', 'g23', 'g24', 'g25', 'g26', 'g27', 'g28', 'g29', 'g30', 'g31', 'g32', 'g33', 'g34', 'g35', 'g36', 'g37', 'g38', 'g39', 'g40', 'g41', 'g42', 'g43', 'g44', 'g45', 'g46', 'g47', 'g48', 'g49', 'g50', 'g51', 'g52', 'g53', 'g54', 'g55', 'g56', 'g57', 'g58', 'g59', 'g60', 'g61', 'g62', 'g63', 'g64', 
#         'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'h8', 'h9', 'h10', 'h11', 'h12', 'h13', 'h14', 'h15', 'h16', 'h17', 'h18', 'h19', 'h20', 'h21', 'h22', 'h23', 'h24', 'h25', 'h26', 'h27', 'h28', 'h29', 'h30', 'h31', 'h32', 'h33', 'h34', 'h35', 'h36', 'h37', 'h38', 'h39', 'h40', 'h41', 'h42', 'h43', 'h44', 'h45', 'h46', 'h47', 'h48', 'h49', 'h50', 'h51', 'h52', 'h53', 'h54', 'h55', 'h56', 'h57', 'h58', 'h59', 'h60', 'h61', 'h62', 'h63', 'h64', 
#         'i1', 'i2', 'i3', 'i4', 'i5', 'i6', 'i7', 'i8', 'i9', 'i10', 'i11', 'i12', 'i13', 'i14', 'i15', 'i16', 'i17', 'i18', 'i19', 'i20', 'i21', 'i22', 'i23', 'i24', 'i25', 'i26', 'i27', 'i28', 'i29', 'i30', 'i31', 'i32', 'i33', 'i34', 'i35', 'i36', 'i37', 'i38', 'i39', 'i40', 'i41', 'i42', 'i43', 'i44', 'i45', 'i46', 'i47', 'i48', 'i49', 'i50', 'i51', 'i52', 'i53', 'i54', 'i55', 'i56', 'i57', 'i58', 'i59', 'i60', 'i61', 'i62', 'i63', 'i64', 
#         'j1', 'j2', 'j3', 'j4', 'j5', 'j6', 'j7', 'j8', 'j9', 'j10', 'j11', 'j12', 'j13', 'j14', 'j15', 'j16', 'j17', 'j18', 'j19', 'j20', 'j21', 'j22', 'j23', 'j24', 'j25', 'j26', 'j27', 'j28', 'j29', 'j30', 'j31', 'j32', 'j33', 'j34', 'j35', 'j36', 'j37', 'j38', 'j39', 'j40', 'j41', 'j42', 'j43', 'j44', 'j45', 'j46', 'j47', 'j48', 'j49', 'j50', 'j51', 'j52', 'j53', 'j54', 'j55', 'j56', 'j57', 'j58', 'j59', 'j60', 'j61', 'j62', 'j63', 'j64']

    tmp_X_test = X_test.to_numpy()
    tmp_X_test = tmp_X_test[20:]
    
    test = test.to_numpy()
    
    test = test[20:,:]
    
    X_index = X_test.index.copy()
    X_index = X_index[20:]
    
    test = pd.DataFrame(test,columns=X_test.columns)
    
    X_test = tmp_X_test.reshape(tmp_X_test.shape[0], 1, tmp_X_test.shape[1])

    merged_invade = merged_invade.to_numpy()
    
    test.index = merged_invade[20:,0]
    
    X_pred = model.predict(X_test)
    X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[2])
    X_pred = pd.DataFrame(X_pred, columns=test.columns)
    X_pred.index = test.index
    
    scored = pd.DataFrame(index=test.index)
    Xtest = X_test.reshape(X_test.shape[0], X_test.shape[2])
    scored['Loss_mae'] = np.mean(np.abs(X_pred-Xtest), axis = 1)
    scored['Threshold'] = 0.085
    scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']
    scored.head()
    print(scored)
    
    cnt = scored.Anomaly.values
    str(cnt)
    unique, cnts = np.unique(cnt,return_counts = True)
    uniq_cnt_dict = dict(zip(unique, cnts))
    
    if True in uniq_cnt_dict :
        print(1)
        if uniq_cnt_dict[True] > 5 :
            send_to_firebase_cloud_messaging()
            print("1")
         
    
    terminate_time = timeit.default_timer() # 종료 시간 체크
    print("%f초 걸렸습니다." % (terminate_time - start_time))
    
    #서버에 맨위 한줄 지우기
#     test = list(ref.get().keys())[0]
#     data = ref.get()
#     for key,val in data.items():
#         if (test == key):
#             delete_user_ref = ref.child(key)
#             delete_user_ref.delete()
#             break;
#         else:
#             print(usernameDoesNotExist)
        
    

       Loss_mae  Threshold  Anomaly
23.0   0.127473      0.085     True
22.75  0.125527      0.085     True
22.5   0.121380      0.085     True
22.75  0.117058      0.085     True
22.75  0.112166      0.085     True
23.0   0.108739      0.085     True
22.0   0.104473      0.085     True
22.75  0.101260      0.085     True
22.75  0.099835      0.085     True
22.75  0.098511      0.085     True
1
Successfully sent message: projects/sensor-3b995/messages/0:1622813901530204%df96c9c4df96c9c4
1
4.066657초 걸렸습니다.
       Loss_mae  Threshold  Anomaly
23.0   0.127473      0.085     True
22.75  0.125527      0.085     True
22.5   0.121380      0.085     True
22.75  0.117058      0.085     True
22.75  0.112166      0.085     True
23.0   0.108739      0.085     True
22.0   0.104473      0.085     True
22.75  0.101260      0.085     True
22.75  0.099835      0.085     True
22.75  0.098511      0.085     True
1
Successfully sent message: projects/sensor-3b995/messages/0:1622813904796908%df96c9c4df96c9

       Loss_mae  Threshold  Anomaly
22.0   0.069187      0.085    False
22.5   0.067617      0.085    False
22.5   0.066779      0.085    False
22.75  0.066967      0.085    False
22.5   0.066959      0.085    False
22.5   0.066877      0.085    False
22.75  0.067726      0.085    False
22.75  0.068229      0.085    False
22.5   0.067405      0.085    False
22.75  0.067532      0.085    False
3.818591초 걸렸습니다.
       Loss_mae  Threshold  Anomaly
22.75  0.066967      0.085    False
22.5   0.066959      0.085    False
22.5   0.066877      0.085    False
22.75  0.067726      0.085    False
22.75  0.068229      0.085    False
22.5   0.067405      0.085    False
22.75  0.067532      0.085    False
22.5   0.069270      0.085    False
23.0   0.073396      0.085    False
22.25  0.074776      0.085    False
3.526360초 걸렸습니다.
       Loss_mae  Threshold  Anomaly
22.75  0.067726      0.085    False
22.75  0.068229      0.085    False
22.5   0.067405      0.085    False
22.75  0.067532      0.085    

       Loss_mae  Threshold  Anomaly
23.5   0.095345      0.085     True
23.0   0.091813      0.085     True
22.75  0.091361      0.085     True
23.5   0.088805      0.085     True
23.25  0.087095      0.085     True
23.5   0.086673      0.085     True
23.0   0.084625      0.085    False
23.0   0.083620      0.085    False
23.0   0.080331      0.085    False
22.75  0.079694      0.085    False
1
Successfully sent message: projects/sensor-3b995/messages/0:1622814027530516%df96c9c4df96c9c4
1
3.377940초 걸렸습니다.
       Loss_mae  Threshold  Anomaly
22.75  0.091361      0.085     True
23.5   0.088805      0.085     True
23.25  0.087095      0.085     True
23.5   0.086673      0.085     True
23.0   0.084625      0.085    False
23.0   0.083620      0.085    False
23.0   0.080331      0.085    False
22.75  0.079694      0.085    False
23.25  0.079489      0.085    False
23.0   0.078691      0.085    False
1
3.281420초 걸렸습니다.
       Loss_mae  Threshold  Anomaly
23.5   0.086673      0.085     True
23

       Loss_mae  Threshold  Anomaly
22.5   0.065104      0.085    False
22.5   0.064871      0.085    False
23.0   0.064913      0.085    False
23.0   0.066301      0.085    False
23.0   0.069369      0.085    False
22.75  0.071456      0.085    False
23.0   0.070055      0.085    False
23.0   0.071079      0.085    False
23.0   0.073376      0.085    False
22.75  0.073706      0.085    False
2.876233초 걸렸습니다.
       Loss_mae  Threshold  Anomaly
23.0   0.064913      0.085    False
23.0   0.066301      0.085    False
23.0   0.069369      0.085    False
22.75  0.071456      0.085    False
23.0   0.070055      0.085    False
23.0   0.071079      0.085    False
23.0   0.073376      0.085    False
22.75  0.073706      0.085    False
23.0   0.073933      0.085    False
23.0   0.074715      0.085    False
2.920881초 걸렸습니다.
       Loss_mae  Threshold  Anomaly
23.0   0.069369      0.085    False
22.75  0.071456      0.085    False
23.0   0.070055      0.085    False
23.0   0.071079      0.085    

       Loss_mae  Threshold  Anomaly
22.5   0.077406      0.085    False
23.25  0.077284      0.085    False
23.0   0.076854      0.085    False
23.25  0.077355      0.085    False
22.75  0.077512      0.085    False
23.0   0.077967      0.085    False
22.75  0.078296      0.085    False
22.25  0.077746      0.085    False
23.5   0.078765      0.085    False
23.25  0.078010      0.085    False
4.208259초 걸렸습니다.
       Loss_mae  Threshold  Anomaly
23.25  0.077355      0.085    False
22.75  0.077512      0.085    False
23.0   0.077967      0.085    False
22.75  0.078296      0.085    False
22.25  0.077746      0.085    False
23.5   0.078765      0.085    False
23.25  0.078010      0.085    False
23.0   0.076964      0.085    False
23.0   0.076799      0.085    False
23.0   0.076197      0.085    False
3.102323초 걸렸습니다.
       Loss_mae  Threshold  Anomaly
23.0   0.077967      0.085    False
22.75  0.078296      0.085    False
22.25  0.077746      0.085    False
23.5   0.078765      0.085    

...

       Loss_mae  Threshold  Anomaly
22.75  0.079592      0.085    False
23.25  0.078714      0.085    False
23.25  0.077893      0.085    False
23.0   0.077351      0.085    False
22.75  0.077048      0.085    False
23.25  0.077519      0.085    False
23.25  0.077775      0.085    False
23.75  0.078356      0.085    False
23.0   0.079229      0.085    False
22.5   0.082220      0.085    False
3.038751초 걸렸습니다.
       Loss_mae  Threshold  Anomaly
23.25  0.077893      0.085    False
23.0   0.077351      0.085    False
22.75  0.077048      0.085    False
23.25  0.077519      0.085    False
23.25  0.077775      0.085    False
23.75  0.078356      0.085    False
23.0   0.079229      0.085    False
22.5   0.082220      0.085    False
22.75  0.082698      0.085    False
22.75  0.082882      0.085    False
2.974839초 걸렸습니다.
       Loss_mae  Threshold  Anomaly
22.75  0.077048      0.085    False
23.25  0.077519      0.085    False
23.25  0.077775      0.085    False
23.75  0.078356      0.085    

       Loss_mae  Threshold  Anomaly
22.75  0.082661      0.085    False
23.0   0.081694      0.085    False
22.75  0.080301      0.085    False
22.75  0.079984      0.085    False
22.5   0.079419      0.085    False
22.5   0.078111      0.085    False
22.75  0.077671      0.085    False
23.25  0.077594      0.085    False
23.0   0.077670      0.085    False
22.75  0.077554      0.085    False
3.392363초 걸렸습니다.
       Loss_mae  Threshold  Anomaly
22.75  0.079984      0.085    False
22.5   0.079419      0.085    False
22.5   0.078111      0.085    False
22.75  0.077671      0.085    False
23.25  0.077594      0.085    False
23.0   0.077670      0.085    False
22.75  0.077554      0.085    False
22.75  0.078070      0.085    False
23.25  0.077646      0.085    False
23.0   0.077365      0.085    False
3.035265초 걸렸습니다.
       Loss_mae  Threshold  Anomaly
22.5   0.078111      0.085    False
22.75  0.077671      0.085    False
23.25  0.077594      0.085    False
23.0   0.077670      0.085    

       Loss_mae  Threshold  Anomaly
23.0   0.106635      0.085     True
22.75  0.109563      0.085     True
23.25  0.111688      0.085     True
23.0   0.113011      0.085     True
22.75  0.114418      0.085     True
23.0   0.114318      0.085     True
23.25  0.111562      0.085     True
22.5   0.104781      0.085     True
23.0   0.096406      0.085     True
23.0   0.092820      0.085     True
1
Successfully sent message: projects/sensor-3b995/messages/0:1622814702061624%df96c9c4df96c9c4
1
2.869952초 걸렸습니다.
       Loss_mae  Threshold  Anomaly
23.0   0.113011      0.085     True
22.75  0.114418      0.085     True
23.0   0.114318      0.085     True
23.25  0.111562      0.085     True
22.5   0.104781      0.085     True
23.0   0.096406      0.085     True
23.0   0.092820      0.085     True
23.0   0.092937      0.085     True
23.5   0.093508      0.085     True
23.0   0.094140      0.085     True
1
Successfully sent message: projects/sensor-3b995/messages/0:1622814704854257%df96c9c4df96c9

       Loss_mae  Threshold  Anomaly
23.0   0.091476      0.085     True
22.75  0.088257      0.085     True
23.5   0.086644      0.085     True
22.75  0.084220      0.085    False
23.0   0.081939      0.085    False
23.0   0.079838      0.085    False
22.75  0.078525      0.085    False
23.0   0.079259      0.085    False
23.25  0.080639      0.085    False
23.0   0.079695      0.085    False
1
3.016141초 걸렸습니다.
       Loss_mae  Threshold  Anomaly
23.5   0.086644      0.085     True
22.75  0.084220      0.085    False
23.0   0.081939      0.085    False
23.0   0.079838      0.085    False
22.75  0.078525      0.085    False
23.0   0.079259      0.085    False
23.25  0.080639      0.085    False
23.0   0.079695      0.085    False
22.75  0.080247      0.085    False
23.75  0.080295      0.085    False
1
3.153561초 걸렸습니다.
       Loss_mae  Threshold  Anomaly
23.0   0.081939      0.085    False
23.0   0.079838      0.085    False
22.75  0.078525      0.085    False
23.0   0.079259      0.085

       Loss_mae  Threshold  Anomaly
23.0   0.082559      0.085    False
23.0   0.084024      0.085    False
23.0   0.085147      0.085     True
22.75  0.085620      0.085     True
23.0   0.087093      0.085     True
22.5   0.087789      0.085     True
23.5   0.089165      0.085     True
23.25  0.089541      0.085     True
23.0   0.089376      0.085     True
23.25  0.089404      0.085     True
1
Successfully sent message: projects/sensor-3b995/messages/0:1622814818487519%df96c9c4df96c9c4
1
3.104735초 걸렸습니다.
       Loss_mae  Threshold  Anomaly
23.0   0.085147      0.085     True
22.75  0.085620      0.085     True
23.0   0.087093      0.085     True
22.5   0.087789      0.085     True
23.5   0.089165      0.085     True
23.25  0.089541      0.085     True
23.0   0.089376      0.085     True
23.25  0.089404      0.085     True
23.0   0.090232      0.085     True
23.25  0.090227      0.085     True
1
Successfully sent message: projects/sensor-3b995/messages/0:1622814821564845%df96c9c4df96c9